# Извлечение эмбеддингов постов с помощью DistilBERT

**Цель ноутбука.**  
Создать текстовые эмбеддинги для постов социальной сети на основе предобученной языковой модели **DistilBERT**, и сократить их размерность методом **PCA** и сохранить результат для последующего обучения модели рекомендаций.

**Входные данные:**  
- `post.csv` — таблица с полями `post_id`, `text`, `topic`.

**Выходные данные:**  
- `embeddings_post.csv` — 20-мерные BERT-эмбеддинги постов (`pca_bert_1`–`pca_bert_20`).

**Этапы:**  
1. Импорт библиотек и загрузка данных.  
2. Токенизация текстов и подготовка датасета.  
3. Извлечение эмбеддингов через `DistilBertModel`.  
4. Снижение размерности до 20 компонент с помощью PCA.  
5. Сохранение итогового датафрейма.

> Все вычисления выполняются без изменения архитектуры модели, используется предобученный чекпойнт `distilbert-base-cased`.


In [ ]:
# Импортируем основные библиотеки для работы с PyTorch, трансформерами и обработкой данных
import torch
from transformers import AutoTokenizer, DistilBertModel
from torch.utils.data import DataLoader, Dataset
from transformers import DataCollatorWithPadding
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from tqdm import tqdm

c:\Users\Администратор\Desktop\START_ML\Karpov_Courses_pet\Deep_learning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Загружаем таблицу постов
post = pd.read_csv("post.csv")
post.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [ ]:
# Инициализируем токенайзер и модель DistilBERT
checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = DistilBertModel.from_pretrained(checkpoint)

# Определяем устройство (GPU, если доступен)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Класс-обёртка для датасета постов, возвращает токенизированный текст
class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        return self.tokenizer(
            self.texts[index],
            padding=False,
            truncation=True
        )

In [ ]:
# Формируем DataLoader для последовательной обработки батчами
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
dataset = PostDataset(post["text"].tolist(), tokenizer)
loader = DataLoader(dataset, batch_size=64, collate_fn=data_collator)

In [ ]:
# Функция для извлечения CLS-эмбеддингов из DistilBERT
@torch.inference_mode()
def get_embeddings(model, loader):
    model.eval()
    embeddings = []

    for batch in tqdm(loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        # Используем [CLS]-токен в качестве представления текста
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings.append(cls_embeddings.cpu())
    
    return torch.cat(embeddings)

In [ ]:
# Извлекаем эмбеддинги для всех постов
embeddings = get_embeddings(model, loader)
print(embeddings.shape)

100%|██████████| 110/110 [00:52<00:00,  2.09it/s]

torch.Size([7023, 768])


In [ ]:
# Снижаем размерность до 20 компонент PCA для последующего обучения модели
pca = PCA(n_components=20)
reduced_embeddings = pca.fit_transform(embeddings)
reduced_embeddings

array([[-0.78907362,  1.57899475,  1.4211591 , ..., -0.03371233,
        -0.58001961, -0.00612047],
       [-0.7927638 ,  1.52105614,  0.89774562, ...,  0.3261879 ,
        -0.13357319, -0.33769165],
       [-0.80129194,  1.22360769,  0.66945213, ..., -0.07861749,
        -0.77339762,  0.17103792],
       ...,
       [ 0.46940024, -0.87485698, -0.48207698, ...,  0.2974304 ,
        -0.17970886,  0.1804096 ],
       [ 1.57979211, -0.44861258, -0.12669208, ..., -0.23992028,
         0.22837455,  0.11805765],
       [ 0.89446924, -0.53490048, -0.19991385, ...,  0.07923052,
         0.44692684,  0.10423737]])

In [ ]:
# Формируем датафрейм с эмбеддингами и id поста
embedding_df = pd.DataFrame(reduced_embeddings, 
                            columns=(f"pca_bert_{i+1}" for i in range(20)))
embedding_df['post_id'] = post['post_id'].values
embedding_df

,pca_bert_1,pca_bert_2,pca_bert_3,pca_bert_4,pca_bert_5,pca_bert_6,pca_bert_7,pca_bert_8,pca_bert_9,pca_bert_10,...,pca_bert_12,pca_bert_13,pca_bert_14,pca_bert_15,pca_bert_16,pca_bert_17,pca_bert_18,pca_bert_19,pca_bert_20,post_id
0,-0.789074,1.578995,1.421159,0.292831,0.196380,0.482332,-0.172336,0.252792,0.128668,-0.029784,...,-0.536405,0.113529,0.143566,0.053079,0.060490,-0.177239,-0.033712,-0.580020,-0.006120,1
1,-0.792764,1.521056,0.897746,-0.228620,0.109591,-0.125362,-0.163190,-0.005947,-0.349708,-0.423531,...,0.690819,0.125367,0.447225,-0.153495,0.281642,0.264169,0.326188,-0.133573,-0.337692,2
2,-0.801292,1.223608,0.669452,-1.323420,-0.081454,-0.550803,0.049824,0.241455,-0.509045,-0.007866,...,-0.182175,0.202675,-0.066136,0.210822,0.072629,-0.023954,-0.078617,-0.773398,0.171038,3
3,-0.867122,0.971222,1.577188,-0.833192,0.841209,0.292207,0.742447,-0.886750,-0.494585,0.055655,...,0.230937,-0.453456,0.290870,0.044735,-0.518971,-0.258113,0.583832,-0.053154,-0.287652,4
4,-0.411659,0.809849,0.682362,-0.790953,-0.186110,0.596705,0.169838,0.758098,-0.193135,0.064067,...,-0.007051,-0.300151,-0.287257,0.124016,-0.170200,-0.403123,-0.233267,0.087435,0.026991,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,0.983509,-0.483879,-0.001963,-0.066758,0.059189,-0.342647,0.026161,0.575154,-0.049793,0.668213,...,0.143204,-0.063735,0.365156,-0.025876,0.116609,-0.394875,0.124753,-0.223166,-0.039118,7315
7019,0.821776,-0.706064,0.164485,0.391023,0.029565,-0.233316,0.336846,0.425709,0.112211,0.308368,...,-0.033479,-0.205864,-0.291056,-0.178710,-0.052942,0.198309,-0.113324,-0.149379,0.052039,7316
7020,0.469400,-0.874857,-0.482077,0.138474,-0.257596,-0.545992,-0.031882,0.287168,-0.504227,0.519209,...,-0.044093,-0.063617,-0.193567,-0.108290,-0.130713,0.403388,0.297430,-0.179709,0.180410,7317
7021,1.579792,-0.448613,-0.126692,-0.100854,0.284518,0.370505,-0.190609,0.049485,-0.189625,0.021933,...,-0.288475,-0.277857,0.454060,-0.071188,0.129983,-0.086229,-0.239920,0.228375,0.118058,7318


In [ ]:
# Сохраняем результат в CSV для использования в следующих этапах
embedding_df.to_csv("embeddings_post.csv", index=False)

## Итоги ноутбука

- Сгенерированы эмбеддинги постов на основе `DistilBERT`.  
- Размерность снижена до 20 компонент с помощью `PCA`.  
- Результат сохранён в файл `embeddings_post.csv`.

**Зачем это нужно:**  
Полученные эмбеддинги использоваться как признаки `post_features` при обучении рекомендательной модели.

**Следующие шаги:**  
- объединить эти эмбеддинги с таблицами `user` и `feed`;  
- обучить модель, предсказывающую вероятность лайка (`target=1`);  
- оценить качество по `Hitrate@5`.
